In [1]:
# Third party libraries
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets

# Import local module
import multyscale

In [2]:
# %% Load example stimulus
stimulus = np.asarray(Image.open("example_stimulus.png").convert("L"))

# %% Parameters of image
shape = stimulus.shape  # filtershape in pixels
# visual extent, same convention as pyplot:
visextent = (-16, 16, -16, 16)


In [3]:
# %% Create model
model = multyscale.models.FLODOG_RHS2007(shape, visextent)

In [4]:
filters_output = model.bank.apply(stimulus)

In [5]:
weighted_outputs = model.weight_outputs(filters_output)

In [29]:
def f(sdmix, spatial_window_scalar):
    model.spatial_window_scalar = spatial_window_scalar
    model.sdmix = sdmix
    model.scale_norm_weights = multyscale.normalization.scale_norm_weights_gaussian(
        len(model.scale_weights), model.sdmix
    )
    model.normalization_weights =       multyscale.normalization.create_normalization_weights(
        6, 7, model.scale_norm_weights, model.orientation_norm_weights
    )
    model.window_sigmas = np.broadcast_to(
        np.array(model.center_sigmas)[None, ..., None], (6, 7, 2)
    )

    print(f"Running {model.sdmix,model.spatial_window_scalar}...")
    output = model.normalize_outputs(weighted_outputs).sum((0,1))

    plt.subplot(1, 2, 1)
    plt.imshow(output, extent=visextent)
    plt.subplot(1, 2, 2)
    plt.plot(output[512, 250:750])
    print("Done!")

ipywidgets.interact_manual(f, sdmix=ipywidgets.FloatSlider(description = "Scale-mixing Gaussian SD:", min = 0.25, max = 3., value = .5, step=0.25), spatial_window_scalar=ipywidgets.FloatSlider(description = "Spatial averaging window scalar:", min = 1, max = 4, value = 2, step=1))

interactive(children=(FloatSlider(value=0.5, description='Scale-mixing Gaussian SD:', max=3.0, min=0.25, step=…

<function __main__.f(sdmix, spatial_window_scalar)>